In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.2 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.7-py2.py3-none-any.whl size=317907716 sha256=cd12f738cf43f24434770034a947a8d72d0eb1721a2d6d8fefe1e042bef5bc85
  Stored in directory: /opt/work/.cache/pip/wheels/90/7b/3a/f4e0b629a2ec0ac23c6ea92ee30d7fb269def367fd9f70bcf9
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("IcebergMinimal")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.rest.type", "rest")
    .config("spark.sql.catalog.rest.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.rest.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.rest.s3.path-style-access", "true")
    .config("spark.sql.catalog.rest.s3.access-key-id", "admin")
    .config("spark.sql.catalog.rest.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.rest.s3.region", "us-east-1")
    .config("spark.sql.defaultCatalog", "rest")
    .getOrCreate()
)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 15:52:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/08 15:52:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.sql("SHOW NAMESPACES IN rest").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
|silver   |
+---------+



In [46]:
spark.sql("SHOW TABLES IN rest.silver").show(truncate=False)

+---------+------------+-----------+
|namespace|tableName   |isTemporary|
+---------+------------+-----------+
|silver   |listings_all|false      |
+---------+------------+-----------+



In [142]:
spark.sql("SELECT COUNT(*) FROM rest.raw.mubawab").show(truncate=False)

+--------+
|count(1)|
+--------+
|2524    |
+--------+



In [143]:
spark.sql("SELECT COUNT(*) FROM rest.raw.avito").show(truncate=False)

+--------+
|count(1)|
+--------+
|3745    |
+--------+



In [234]:
spark.sql("TRUNCATE TABLE rest.silver.listings_all").show(truncate=False)

++
||
++
++



In [232]:
## spark.sql("DROP TABLE rest.silver.listings_all").show(truncate=False)

++
||
++
++



In [233]:
spark.sql("SELECT COUNT(*) FROM rest.silver.listings_all").show(truncate=False)

[Stage 558:>                                                        (0 + 1) / 1]

+--------+
|count(1)|
+--------+
|0       |
+--------+



In [146]:
silver = spark.sql("SELECT * FROM rest.silver.listings_all")

In [147]:
silver.select("url").distinct().count()

3436

In [148]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number

# keep latest per URL
w = Window.partitionBy("url").orderBy(col("ingest_ts").desc())

silver_dedup = (
    silver
    .withColumn("rn", row_number().over(w))
    .filter(col("rn") == 1)
    .drop("rn")
)

# verify
print("After de-duplication:", silver_dedup.count())


After de-duplication: 3436


In [149]:
from pyspark.sql.functions import col, count

silver.groupBy("site") \
      .agg(count("*").alias("count")) \
      .orderBy(col("count").desc()) \
      .show(100, truncate=False)

+-------+-----+
|site   |count|
+-------+-----+
|avito  |3991 |
|mubawab|2341 |
+-------+-----+



In [150]:
silver.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- error: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- site: string (nullable = false)
 |-- offre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description_text: string (nullable = true)
 |-- offre_match: boolean (nullable = true)
 |-- surface_habitable: string (nullable = true)
 |-- caution: string (nullable = true)
 |-- zoning: string (nullable = true)
 |-- type_d_appartement: string (nullable = true)
 |-- standing: string (nullable = true)
 |-- s

In [151]:
silver.limit(1).toPandas()

,id,url,error,ingest_ts,site,offre,price,title,seller,published_date,...,statut_du_terrain,surface_de_la_parcelle,type_du_sol,etage_du_bien,annees,constructibilite,livraison,orientation,etat,nombre_d_etages
0,8244102,https://www.mubawab.ma/fr/a/8244102/terrain-%C...,None,2025-11-08 16:02:30.003,mubawab,vente,800000.0,Terrain à vendre à Route de Fez. Surface de 40...,elhajouy immo,None,...,None,None,None,None,None,None,Titré,None,None,None


## Price

In [75]:
silver.select("price").distinct().show(10)

+---------+
|    price|
+---------+
| 300000.0|
| 330000.0|
|   4800.0|
|   1.66E7|
|  18400.0|
| 995000.0|
|  15750.0|
|1100000.0|
| 165000.0|
|  16800.0|
+---------+
only showing top 10 rows



In [152]:
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType

# overwrite the same column, no new dataset
silver = silver.withColumn("price", col("price").cast(DecimalType(38, 0)))

# verify
silver.select("price").distinct().orderBy(col("price").desc()).show(5)
silver.select("price").distinct().orderBy(col("price").asc()).show(5)

+----------+
|     price|
+----------+
|1500000000|
| 650000000|
| 410000000|
| 230000000|
| 162500000|
+----------+
only showing top 5 rows

+-----+
|price|
+-----+
| NULL|
|  125|
|  130|
|  200|
|  249|
+-----+
only showing top 5 rows



In [77]:
silver.select("ingest_ts").orderBy(col("ingest_ts")).show(10 , truncate=False)

+-----------------------+
|ingest_ts              |
+-----------------------+
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:20.003|
|2025-11-08 15:56:30.007|
+-----------------------+
only showing top 10 rows



In [153]:
from pyspark.sql import functions as F

silver = silver.withColumn(
    "published_date",
    F.when(F.col("published_date").isNull(), F.col("ingest_ts"))
     .otherwise(F.col("published_date"))
)

In [154]:
silver.select(
    F.count(F.when(F.col("published_date").isNull(), 1)).alias("null_published"),
    F.count("*").alias("total")
).show()


+--------------+-----+
|null_published|total|
+--------------+-----+
|             0| 6332|
+--------------+-----+



In [80]:
silver.select("id", "site" , "published_date").show(20, truncate=False)

+-------+-------+-----------------------+
|id     |site   |published_date         |
+-------+-------+-----------------------+
|8245961|mubawab|2025-11-08 16:54:50.005|
|8245969|mubawab|2025-11-08 16:54:50.005|
|8245971|mubawab|2025-11-08 16:54:50.005|
|8245980|mubawab|2025-11-08 16:54:50.005|
|8245987|mubawab|2025-11-08 16:54:40.003|
|8245989|mubawab|2025-11-08 16:54:40.003|
|8245992|mubawab|2025-11-08 16:54:40.003|
|8245997|mubawab|2025-11-08 16:54:40.003|
|8246003|mubawab|2025-11-08 16:54:30.003|
|8246008|mubawab|2025-11-08 16:54:30.003|
|8246009|mubawab|2025-11-08 16:54:30.003|
|8246010|mubawab|2025-11-08 16:54:30.003|
|8246020|mubawab|2025-11-08 16:54:20.002|
|8246023|mubawab|2025-11-08 16:54:20.002|
|8246024|mubawab|2025-11-08 16:54:20.002|
|8246029|mubawab|2025-11-08 16:54:20.002|
|8246044|mubawab|2025-11-08 16:54:20.002|
|8246045|mubawab|2025-11-08 16:54:10.005|
|8246057|mubawab|2025-11-08 16:54:10.005|
|8246069|mubawab|2025-11-08 16:54:10.005|
+-------+-------+-----------------

In [81]:
from pyspark.sql.functions import col, count

silver.groupBy("city") \
      .agg(count("*").alias("count")) \
      .orderBy(col("count").asc()) \
      .show(10, truncate=False)

+--------------+-----+
|city          |count|
+--------------+-----+
|Jouamaa       |1    |
|Inchaden      |1    |
|Ziaida        |1    |
|Dakhla        |1    |
|Agadir Melloul|1    |
|Imi Ouaddar   |1    |
|Tnine Aglou   |1    |
|Laäyoune      |1    |
|Oulad Dahou   |1    |
|Oulad Frej    |1    |
+--------------+-----+
only showing top 10 rows



In [155]:
import re, unicodedata
from pyspark.sql.functions import col, trim, udf
from pyspark.sql.types import StringType

def _unaccent(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def _clean_base(s: str) -> str:
    s = _unaccent(s or "").lower()
    s = re.sub(r"[^a-z0-9\s'\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

ARABIC = {
    "الدار البيضاء": "casablanca",
    "مراكش": "marrakech",
    "طنجة": "tanger",
    "فاس": "fes",
    "وجدة": "oujda",
    "القنيطرة": "kenitra",
    "العيون": "laayoune",
    "الرباط": "rabat",
    "سلا": "sale",
    "تطوان": "tetouan",
}

ALIASES = {
    "casa": "casablanca", "casa blanca": "casablanca", "casablance": "casablanca",
    "marrakesh": "marrakech",
    "tangier": "tanger",
    "fez": "fes",
    "kénitra": "kenitra", "kenitra": "kenitra",
    "meknès": "meknes",
    "tétouan": "tetouan",
    "béni mellal": "beni mellal",
    "laâyoune": "laayoune", "laäyoune": "laayoune", "laayoun": "laayoune",
    "m'diq": "mdiq", "m diq": "mdiq",
    "dcheïra el jihadia": "dcheira el jihadia",
    "inzegan": "inezgane", "inezgane": "inezgane",
    "ait melloul": "ait melloul",
    "dar bouazza": "dar bouazza",
    "bouskoura": "bouskoura",
    "harhoura": "harhoura",
    "skhirat": "skhirat",
    "tamesna": "tamesna",
    "rabat": "rabat",
    "sale": "sale",
}

NULLY = {"", "null", "none", "n/a", "nan", "tout le maroc", "toute la ville"}

def normalize_city_str(val: str) -> str:
    if val is None:
        return None
    if val in ARABIC:
        return ARABIC[val]
    base = _clean_base(val)
    if base in NULLY:
        return None
    # direct alias
    if base in ALIASES:
        return ALIASES[base]
    # regex-based cleanup
    base = base.replace("m diq", "mdiq").replace("m' diq", "mdiq")
    if base in ALIASES:
        return ALIASES[base]
    return base or None

normalize_city_udf = udf(normalize_city_str, StringType())

# overwrite same column
silver = silver.withColumn("city", normalize_city_udf(trim(col("city"))))

# verify
silver.groupBy("city").count().orderBy(col("count").desc()).show(10, truncate=False)

+----------+-----+
|city      |count|
+----------+-----+
|casablanca|2017 |
|marrakech |984  |
|rabat     |643  |
|tanger    |613  |
|mohammedia|231  |
|kenitra   |202  |
|agadir    |174  |
|temara    |172  |
|bouskoura |158  |
|fes       |90   |
+----------+-----+
only showing top 10 rows



In [156]:
silver.select("city").distinct().count()

120

In [157]:
from pyspark.sql.functions import col, count

silver.groupBy("neighborhood") \
      .agg(count("*").alias("count")) \
      .orderBy(col("count").asc()) \
      .show(10, truncate=False)

+----------------+-----+
|neighborhood    |count|
+----------------+-----+
|Fouarate        |1    |
|Extension Dakhla|1    |
|Al Filline      |1    |
|Hoda            |1    |
|Jawhara         |1    |
|Hay Inara       |1    |
|Azib Haj Kaddour|1    |
|Les Orangers    |1    |
|Hay Inbiat      |1    |
|Technopolis     |1    |
+----------------+-----+
only showing top 10 rows



In [158]:
import unicodedata, re
from pyspark.sql.functions import col, trim, udf
from pyspark.sql.types import StringType

# --- helpers
def _fix_mojibake(s: str) -> str:
    # common UTF-8 mojibake seen in scraped French/Arabic names
    # (keep set short + pragmatic)
    repl = {
        "Ã©": "é", "Ã¨": "è", "Ãª": "ê", "Ã«": "ë",
        "Ã¢": "â", "Ã¤": "ä", "Ã ": "à", "Ã¹": "ù",
        "Ã»": "û", "Ã¼": "ü", "Ã¯": "ï", "Ã®": "î",
        "Ã´": "ô", "Ã§": "ç",
        "Ã¶": "ö", "Ã±": "ñ",
        "Â°": "°", "Â’": "’", "Â«": "«", "Â»": "»",
        "Â·": "·", "Â": "",  # stray
    }
    for k,v in repl.items():
        s = s.replace(k, v)
    return s

def _unaccent(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

NULLY = {
    "", "null", "none", "n/a", "nan",
    "—", "-", ".", "_", "?", "s/o", "n.c", "nc"
}

# Canonical token normalizers (apply after lowercasing)
TOKEN_FIXES = [
    (r"\bain\b", "ain"),
    (r"\baïn\b", "ain"),
    (r"\bha[iy]\b", "hay"),
    (r"\bhayy\b", "hay"),
    (r"\bz\.?\s*ind(ustrielle)?\b", "zone industrielle"),
    (r"\bzone ind(ustrielle)?\b", "zone industrielle"),
    (r"\bcentre[-\s]?ville\b", "centre ville"),
    (r"\bville[-\s]?nouvelle\b", "ville nouvelle"),
    (r"\bquartier[-\s]administratif\b", "quartier administratif"),
    (r"\bquartier[-\s]du[-\s]parc\b", "quartier du parc"),
    (r"\bquartier[-\s]des[-\s]h[oô]pitaux\b", "quartier des hopitaux"),
    (r"\b2\s*mars\b", "2 mars"),
    (r"\bcasablanca\s*marina\b", "casablanca marina"),
    (r"\broute\s*d['’]\s*", "route de "),
    (r"\bav\s+", "avenue "),
    (r"\bav\.?\b", "avenue"),
    (r"\bbd\b", "boulevard"),
    (r"\bval\s*d['’]or\b", "val d'or"),
    (r"\bp[ao]lmeraie\b", "palmeraie"),
]

# Big alias map (lowercase & unaccented keys -> canonical LOWERCASE)
ALIASES = {
    # high-frequency buckets in your printout
    "centre ville": "centre ville",
    "centre": "centre",
    "centre  ville": "centre ville",
    "centre-ville": "centre ville",

    "les princesses": "les princesses",
    "les princesses ": "les princesses",
    "princesses": "les princesses",
    "les princesses (maarif)": "les princesses",

    "val fleuri": "val fleuri",
    "val  fleuri": "val fleuri",

    "racine": "racine",
    "racine extension": "racine extension",

    "gueliz": "gueliz",
    "guéliz": "gueliz",

    "agdal": "agdal",
    "hay riad": "hay riad",
    "souissi": "souissi",

    "ain diab": "ain diab",
    "aïn diab": "ain diab",

    "ain sebaa": "ain sebaa",
    "aïn sebaa": "ain sebaa",

    "maarif": "maarif",
    "maârif": "maarif",
    "maarif extension": "maarif extension",
    "maârif extension": "maarif extension",

    "californie": "californie",
    "bordeaux": "bordeaux",

    "riviéra": "riviera",
    "riviera": "riviera",

    "oasis": "oasis",
    "oasis sud": "oasis sud",

    "anfa": "anfa",
    "anfa superieur": "anfa superieur",
    "anfa supérieur": "anfa superieur",

    "belvedere": "belvedere",
    "belvédère": "belvedere",

    "de la plage": "de la plage",
    "la gironde": "la gironde",
    "hay hassani": "hay hassani",
    "hay mohammadi": "hay mohammadi",
    "sidi maarouf": "sidi maarouf",
    "sidi bernoussi": "sidi bernoussi",
    "roch es noires": "roches noires",
    "roches noires": "roches noires",
    "quartier des hopitaux": "quartier des hopitaux",
    "quartier des hôpitaux": "quartier des hopitaux",
    "quartier du parc": "quartier du parc",
    "ferme bretonne (hay arraha)": "ferme bretonne",
    "ferme bretonne": "ferme bretonne",
    "pal mier": "palmier",
    "palmier": "palmier",
    "beausejour": "beausejour",
    "beauséjour": "beausejour",
    "bourn gougne": "bourgogne",
    "bourgogne": "bourgogne",
    "bourgogne est": "bourgogne est",
    "bourgogne ouest": "bourgogne ouest",
    "casablanca finance city": "casablanca finance city",
    "cfc": "casablanca finance city",
    "gaut hier": "gauthier",
    "gauthier": "gauthier",
    "triangle d'or": "triangle d'or",

    "hivernage": "hivernage",
    "ennakhil (palmeraie)": "ennakhil (palmeraie)",
    "palmeraie": "palmeraie",
    "targa": "targa",
    "route de l ourika": "route de l'ourika",
    "route d ourika": "route de l'ourika",
    "route d'ourika": "route de l'ourika",
    "route de l'ourika": "route de l'ourika",
    "route de tahanaoute": "route de tahanaoute",
    "route de fez": "route de fez",
    "route de fès": "route de fez",
    "route d amezmiz": "route d'amezmiz",
    "route d'amezmiz": "route d'amezmiz",
    "sem lalia": "semlalia",
    "semlalia": "semlalia",
    "amirich": "amerchich",
    "amerchich": "amerchich",
    "majorelle": "majorelle",
    "massira": "massira",
    "massira 1": "massira 1",
    "massira 2": "massira 2",
    "massira 3": "massira 3",
    "daoudiate": "daoudiate",
    "taghazout": "taghazout",
    "bab doukkala": "bab doukkala",
    "bab atlas": "bab atlas",
    "ourika": "ourika",

    "agadir founty": "founty",
    "founty": "founty",
    "haut founty": "haut founty",
    "haut-founty": "haut founty",
    "quartier administratif": "quartier administratif",
    "talborjt": "talborjt",
    "sonaba": "sonaba",
    "inzegane": "inezgane",
    "inezgane": "inezgane",
    "beni makada lakdima": "beni makada lakdima",
    "beni makada": "beni makada",
    "mghar gha": "mghogha",
    "mghogha": "mghogha",
    "achakar": "achakar",
    "tanger city center": "tanger city center",
    "malabata": "malabata",
    "marchan": "marchan",
    "ziaten": "ziaten",
    "casabarata": "casabarata",
    "branes 2": "branes 2",
    "branes kdima": "branes kdima",
    "m'nar": "mnar",
    "mnar": "mnar",

    "hay riad extension": "hay riad extension",
    "hassan - centre ville": "hassan - centre ville",
    "hassan": "hassan",
    "quartier de l ocean": "quartier de l'ocean",
    "quartier de l'océan": "quartier de l'ocean",
    "quartier de l'ocean": "quartier de l'ocean",
    "ouden a": "ouedayas",
    "oudayas": "oudayas",
    "guich oudaya": "guich oudaya",
    "akkari": "aakkari",
    "aakkari": "aakkari",
    "ya coub el mansour": "yacoub el mansour",
    "yacoub el mansour": "yacoub el mansour",

    "la siesta": "la siesta",
    "el mansouria": "el mansouria",
    "had soualem": "had soualem",
    "deroua": "deroua",
    "nouaceur": "nouaceur",
    "wifak": "wifak",
    "wifaq": "wifak",
    "sidi rahal chatai": "sidi rahal chatai",
    "plage oued cherrat": "plage oued cherrat",

    # generic forms
    "ville nouvelle": "ville nouvelle",
    "nouvelle ville": "ville nouvelle",
    "ancienne medina": "ancienne medina",
    "medina": "medina",
    "médina": "medina",
    "la medina": "medina",
    "ancienne médina": "ancienne medina",

    # other one-offs from your list
    "beausite": "beausite",
    "val d or": "val d'or",
    "c i l": "cil",
    "c.i.l": "cil",
    "cil (hay salam)": "cil",
    "hay salam": "hay salam",
    "ain chock": "ain chock",
    "aïn chock": "ain chock",
    "sidi moumen": "sidi moumen",
    "derb ghalef": "derb ghalef",
    "derb omar": "derb omar",
    "2 mars": "2 mars",
    "hassan centre ville": "hassan - centre ville",
    "el menzeh": "el menzeh",
    "el mansouria": "el mansouria",
    "el haouzia": "el haouzia",
    "laimoune": "laymoune",
    "laymoune": "laymoune",
    "ain sebaa ": "ain sebaa",
    "ain sebaa extension": "ain sebaa",
    "ain diab extension": "ain diab extension",
    "casablanca marina": "casablanca marina",
    "mozar t": "mozart",
    "mozart": "mozart",
    "rabat asfan": "rabat asfan",  # keep literal since unclear
}

def _canon(s: str) -> str:
    if s is None:
        return None
    s = str(s).strip()
    if s.lower() in NULLY:
        return None

    # fix mojibake early
    s = _fix_mojibake(s)

    # normalize punctuation/spaces
    s = re.sub(r"[‐-–—]+", "-", s)          # various dashes -> hyphen
    s = re.sub(r"[’']", "’", s)              # unify apostrophes
    s = re.sub(r"\s+", " ", s).strip()

    # lower + unaccented key for alias lookup
    key = _unaccent(s).lower()

    # token-level fixes
    for pat, rep in TOKEN_FIXES:
        key = re.sub(pat, rep, key)

    key = re.sub(r"[-_/]+", " ", key)
    key = re.sub(r"\s+", " ", key).strip()

    # alias map first
    if key in ALIASES:
        return ALIASES[key]

    # generic cleanup rules (don’t over-aggressively collapse semantics)
    # 1) collapse double words (e.g., "centre  ville")
    key = re.sub(r"\b(\w+)\s+\1\b", r"\1", key)

    # 2) trim generic prefixes/suffixes noise around numbers like "hay tarik 1"
    key = re.sub(r"\b0*(\d+)\b", r"\1", key)

    # 3) stabilize "route de xxx"
    key = re.sub(r"\broute\s+de\s+de\b", "route de", key)

    # return canonical lowercase
    return key or None

normalize_neighborhood_udf = udf(_canon, StringType())

# --- overwrite `neighborhood`
silver = silver.withColumn("neighborhood", normalize_neighborhood_udf(trim(col("neighborhood"))))

In [159]:
from pyspark.sql.functions import col, count

(silver.groupBy("neighborhood")
       .agg(count("*").alias("cnt"))
       .orderBy(col("cnt").asc(), col("neighborhood").asc())
       .show(10, truncate=False))


+------------------+---+
|neighborhood      |cnt|
+------------------+---+
|abouab sala       |1  |
|agadir melloul    |1  |
|ain attig         |1  |
|al aaroui         |1  |
|al amal           |1  |
|al anbar          |1  |
|al filline        |1  |
|al madina aljadida|1  |
|al mostakbal      |1  |
|al moustakbal     |1  |
+------------------+---+
only showing top 10 rows



In [185]:
from pyspark.sql.functions import col, count

silver.groupBy("site" , "property_type") \
      .agg(count("*").alias("count")) \
      .orderBy(col("count").asc()) \
      .show(1000, truncate=False)

+-------+------------------+-----+
|site   |property_type     |count|
+-------+------------------+-----+
|avito  |NULL              |5    |
|avito  |Autre Immobilier  |56   |
|mubawab|Maisons           |75   |
|mubawab|Local commercial  |117  |
|avito  |Maisons           |118  |
|mubawab|Bureaux           |125  |
|mubawab|Terrains et fermes|194  |
|avito  |Terrains et fermes|278  |
|mubawab|Villas et Riads   |363  |
|avito  |Bureaux           |364  |
|avito  |Local             |403  |
|avito  |Villas et Riads   |431  |
|mubawab|Appartements      |1467 |
|avito  |Appartements      |2336 |
+-------+------------------+-----+



In [161]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim
from pyspark.sql.types import StringType
import unicodedata, re

def _unaccent(s: str) -> str:
    if s is None:
        return None
    return "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )

def normalize_property_type(value: str) -> str:
    if value is None:
        return None

    raw = value.strip()
    if raw == "":
        return None

    s = _unaccent(raw).strip().lower()
    s = re.sub(r"\s+", " ", s)

    # ======= mapping demandé =======
    # Ferme + Terrain + Terrains et fermes → Terrains et fermes
    if s in {"ferme", "terrain", "terrains et fermes"}:
        return "Terrains et fermes"

    # Riad + Villa + Villas et Riads → Villas et Riads
    if s in {"riad", "villa", "villas et riads"}:
        return "Villas et Riads"

    # Bureau + Bureaux → Bureaux
    if s in {"bureau", "bureaux"}:
        return "Bureaux"

    # Maison + Maisons → Maisons
    if s in {"maison", "maisons"}:
        return "Maisons"

    # Appartement + Appartements → Appartements
    if s in {"appartement", "appartements"}:
        return "Appartements"

    # valeurs vides / null équivalentes
    if s in {"", "null", "none"}:
        return None

    # pour tout le reste : on garde la valeur originale trimée
    return raw

normalize_property_type_udf = F.udf(normalize_property_type, StringType())

# 🔁 overwrite de la **même** colonne dans le même DataFrame
silver = silver.withColumn(
    "property_type",
    normalize_property_type_udf(trim(col("property_type")))
)

# (optionnel) vérifier le résultat
(
    silver.groupBy("property_type")
          .agg(F.count("*").alias("count"))
          .orderBy(F.col("count").desc())
          .show(100, truncate=False)
)

+------------------+-----+
|property_type     |count|
+------------------+-----+
|Appartements      |3803 |
|Villas et Riads   |794  |
|Bureaux           |489  |
|Terrains et fermes|472  |
|Local             |403  |
|Maisons           |193  |
|Local commercial  |117  |
|Autre Immobilier  |56   |
|NULL              |5    |
+------------------+-----+



In [162]:
from pyspark.sql.functions import col, count

silver.groupBy("equipments") \
      .agg(count("*").alias("count")) \
      .orderBy(col("count").asc()) \
      .show(10, truncate=False)

+-------------------------------------------------------------------+-----+
|equipments                                                         |count|
+-------------------------------------------------------------------+-----+
|[Ascenseur, Balcon, Climatisation, Concierge, Parking]             |1    |
|[Ascenseur, Climatisation, Cuisine équipée, Parking, Sécurité]     |1    |
|[Ascenseur, Concierge, Cuisine équipée, Meublé, Parking]           |1    |
|[Climatisation, Cuisine équipée, Meublé, Parking, Sécurité]        |1    |
|[Duplex, Climatisation, Cuisine équipée, Sécurité, Terrasse]       |1    |
|[Balcon, Cuisine équipée, Meublé]                                  |1    |
|[Ascenseur, Chauffage, Climatisation, Concierge, Parking, Terrasse]|1    |
|[Ascenseur, Balcon, Cuisine équipée, Meublé, Parking, Sécurité]    |1    |
|[Ascenseur, Balcon, Concierge, Meublé, Parking, Terrasse]          |1    |
|[Ascenseur, Balcon, Cuisine équipée]                               |1    |
+-----------

In [163]:
# --- NORMALIZE `equipments` IN-PLACE (silver DataFrame) ---

from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
import re, unicodedata
from typing import Optional


# 1) Canonical vocabulary you want to keep inside `equipments`
CANON = {
    # état / disponibilité / standing / type / foncier
    "neuf","bon_etat","a_renover","livraison_immediate",
    "economique","moyen_standing","haut_standing",
    "duplex","maison","villa","immeuble",
    "loti","titre_foncier","agricole","industriel","service_public",

    # équipements / extérieurs / gestion
    "ascenseur","balcon","terrasse","jardin","garage","parking","piscine",
    "climatisation","chauffage","cuisine_equipee","meuble",
    "securite","concierge","cablage_telephonique",
}

# 2) Synonyms / variants → canonical
SYNONYMS = {
    # état
    "bon etat":"bon_etat","etat correct":"bon_etat","etat-bon":"bon_etat",
    "a renover":"a_renover","à renover":"a_renover","à rénover":"a_renover",
    # disponibilité
    "immediate":"livraison_immediate","immédiate":"livraison_immediate","livraison immediate":"livraison_immediate",
    # standing
    "standing moyen":"moyen_standing","moyen-standing":"moyen_standing","moyen standing":"moyen_standing",
    "haut standing":"haut_standing","tres haut standing":"haut_standing","très haut standing":"haut_standing",
    # type
    "appartement duplex":"duplex","duplexe":"duplex",
    # foncier / zonage / titre
    "titre":"titre_foncier","titré":"titre_foncier","titre foncier":"titre_foncier",
    "lotis":"loti","lotissement":"loti",
    "agricole":"agricole","industriel":"industriel","service public":"service_public",
    # équipements
    "clim":"climatisation","climatisations":"climatisation",
    "chauffages":"chauffage",
    "cuisine equipee":"cuisine_equipee","cuisine equipée":"cuisine_equipee","cuisine équipée":"cuisine_equipee",
    "meuble":"meuble","meublé":"meuble","meublee":"meuble",
    "securite":"securite","sécurité":"securite",
    "concierge":"concierge",
    "cablage telephonique":"cablage_telephonique","câblage telephonique":"cablage_telephonique","câblage téléphonique":"cablage_telephonique",
    "ascenseur":"ascenseur","ascenseurs":"ascenseur",
    "balcon":"balcon","balcons":"balcon",
    "terrasse":"terrasse","terrasses":"terrasse",
    "jardin":"jardin","jardins":"jardin",
    "garage":"garage","garages":"garage",
    "parking":"parking","parkings":"parking",
    "piscine":"piscine","piscines":"piscine",
    # types de bien courants dans tes données
    "maison":"maison","villa":"villa","immeuble":"immeuble",
    # autres graphies fréquentes vues
    "economique":"economique","moyen standing":"moyen_standing","haut-standing":"haut_standing",
}

# 3) helpers
def _strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def _clean_token(t: str) -> str:
    t = _strip_accents(t or "").lower()
    t = re.sub(r"[,_/|;+]", " ", t)        # unify separators
    t = re.sub(r"\s+", " ", t).strip()
    return t

# Optional: lightweight fuzzy match using plain Python Levenshtein
# (fast enough for short token lists; keeps your single-column constraint)
def _levenshtein(a: str, b: str) -> int:
    if a == b: return 0
    if not a: return len(b)
    if not b: return len(a)
    prev = list(range(len(b)+1))
    for i, ca in enumerate(a, 1):
        cur = [i]
        for j, cb in enumerate(b, 1):
            ins = cur[j-1] + 1
            dele = prev[j] + 1
            sub = prev[j-1] + (ca != cb)
            cur.append(min(ins, dele, sub))
        prev = cur
    return prev[-1]

def _to_canonical(tok: str) -> Optional[str]:
    if not tok:
        return None
    # exact map via synonyms
    if tok in SYNONYMS:
        return SYNONYMS[tok]
    # already canonical?
    if tok in CANON:
        return tok
    # try joining words with underscore if looks like "cuisine equipee"
    tok_und = tok.replace(" ", "_")
    if tok_und in CANON: 
        return tok_und
    # fuzzy to nearest canonical if very close (typos)
    # threshold: <=1 for short tokens, <=2 otherwise
    best = None
    best_d = 10**9
    for c in CANON:
        d = _levenshtein(tok_und, c)
        if d < best_d:
            best, best_d = c, d
    if best is not None:
        thr = 1 if len(tok_und) <= 6 else 2
        if best_d <= thr:
            return best
    return None  # drop unknowns (stays within same column contract)

@udf(ArrayType(StringType()))
def normalize_equipments(arr):
    # Accept array<string> or a single string (comma/semicolon separated)
    if arr is None:
        return None  # preserve NULL as NULL (distinct from [])
    tokens = []
    if isinstance(arr, list):
        for x in arr:
            if x is None: 
                continue
            # some inputs can be "A, B, C" inside a single element
            parts = re.split(r"[;,]", str(x))
            tokens.extend(parts)
    else:
        tokens = re.split(r"[;,]", str(arr))

    out = []
    seen = set()
    for t in tokens:
        t = _clean_token(t)
        if not t:
            continue
        can = _to_canonical(t)
        if can and can not in seen:
            seen.add(can)
            out.append(can)

    # stable order: alphabetical for determinism
    out.sort()
    return out

# 4) apply in-place on silver
silver = silver.withColumn("equipments", normalize_equipments(col("equipments")))

In [164]:
from pyspark.sql.functions import col, count

silver.groupBy("site", "surface_habitable" , "surface_totale") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").desc(), col("count").asc()) \
      .show(10, truncate=False)

+-------+-----------------+--------------+-----+
|site   |surface_habitable|surface_totale|count|
+-------+-----------------+--------------+-----+
|mubawab|NULL             |NULL          |2341 |
|avito  |25               |NULL          |1    |
|avito  |NULL             |29            |1    |
|avito  |118              |NULL          |1    |
|avito  |214              |NULL          |1    |
|avito  |120              |160           |1    |
|avito  |160              |160           |1    |
|avito  |50               |55            |1    |
|avito  |300              |500           |1    |
|avito  |88               |NULL          |1    |
+-------+-----------------+--------------+-----+
only showing top 10 rows



In [165]:
from pyspark.sql.functions import col, count

silver.groupBy("site", "surface_totale") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").desc(), col("count").asc()) \
      .show(10, truncate=False)

+-------+--------------+-----+
|site   |surface_totale|count|
+-------+--------------+-----+
|mubawab|NULL          |2341 |
|avito  |2800          |1    |
|avito  |198           |1    |
|avito  |1400          |1    |
|avito  |2343          |1    |
|avito  |165           |1    |
|avito  |790000        |1    |
|avito  |426           |1    |
|avito  |123           |1    |
|avito  |29            |1    |
+-------+--------------+-----+
only showing top 10 rows



In [166]:
from pyspark.sql.functions import col, when, regexp_replace, trim

# 1) Nettoyer/typer surface_de_la_parcelle (souvent "520 m²", espaces, NBSP, etc.)
# - enlève tout sauf chiffres . et ,
# - remplace virgule par point
# - cast en double
parcelle_clean = (
    regexp_replace(trim(col("surface_de_la_parcelle")), r"\u00A0", " ")       # NBSP -> espace
)
parcelle_clean = regexp_replace(parcelle_clean, r"[^0-9\.,]", "")             # garder chiffres . ,
parcelle_clean = regexp_replace(parcelle_clean, r",", ".")                    # virgule -> point
parcelle_clean = parcelle_clean.cast("double")

silver = silver.withColumn("surface_de_la_parcelle", parcelle_clean)

# 2) Overwrite de surface_totale avec parcelle quand surface_totale est NULL et parcelle NON NULL/positive
silver = silver.withColumn(
    "surface_totale",
    when(
        col("surface_totale").isNull() & col("surface_de_la_parcelle").isNotNull() & (col("surface_de_la_parcelle") > 0),
        col("surface_de_la_parcelle")
    ).otherwise(col("surface_totale"))
)

# 3) Drop définitif de surface_de_la_parcelle
silver = silver.drop("surface_de_la_parcelle")

In [167]:
from pyspark.sql.functions import col, when, lower, regexp_replace, trim, coalesce

# helpers
def _num(colname):
    # strip NBSP, keep digits/., replace comma, cast
    c = regexp_replace(trim(col(colname)), r"\u00A0", " ")
    c = regexp_replace(c, r"[^0-9\.,]", "")
    c = regexp_replace(c, r",", ".").cast("double")
    return when(c <= 0, None).otherwise(c)

# A) parse
silver = silver.withColumn("surface_totale", _num("surface_totale")) \
               .withColumn("surface_habitable", _num("surface_habitable"))

# C) type buckets
pt = lower(coalesce(col("property_type"), col("type_de_bien")))

is_land   = pt.rlike(r"(terrain|ferme|terrains|loti|lot|agricole)")
is_appt   = pt.rlike(r"(appart|studio|duplex)")
is_house  = pt.rlike(r"(maison|villa|riad)")
is_office = pt.rlike(r"(bureau|local)")

def keep_between(c, lo, hi): 
    return when((c >= lo) & (c <= hi), c)

# Copy for editing
ST = col("surface_totale")
SH = col("surface_habitable")

# Appartements
ST_appt = keep_between(ST, 15, 1500)
SH_appt = keep_between(SH, 10, 1200)

# Maisons/Villas
ST_house = keep_between(ST, 30, 50000)
SH_house = keep_between(SH, 20, 2000)

# Bureaux/Locaux
ST_off   = keep_between(ST, 10, 50000)
SH_off   = keep_between(SH, 10, 10000)

# Terrains
ST_land = keep_between(ST, 10, 1_000_000)
# land habitable should be null
SH_land = None

silver = silver.withColumn(
    "surface_totale",
    when(is_appt, ST_appt)
    .when(is_house, ST_house)
    .when(is_office, ST_off)
    .when(is_land, ST_land)
    .otherwise(keep_between(ST, 10, 200000))  # fallback conservative
)

silver = silver.withColumn(
    "surface_habitable",
    when(is_appt, SH_appt)
    .when(is_house, SH_house)
    .when(is_office, SH_off)
    .when(is_land, None)  # drop habitable on land
    .otherwise(keep_between(SH, 10, 5000))
)

# D) swap if obviously inverted
ST = col("surface_totale"); SH = col("surface_habitable")
swap_cond = (ST.isNotNull() & SH.isNotNull() &
             (SH >= 2*ST) &
             ((ST <= 400) | (SH / ST >= 5))
            )

silver = silver.withColumn(
    "surface_totale",
    when(swap_cond, SH).otherwise(col("surface_totale"))
).withColumn(
    "surface_habitable",
    when(swap_cond, ST).otherwise(col("surface_habitable"))
)

# E) enforce relationship for non-land
ST = col("surface_totale"); SH = col("surface_habitable")
too_big = (SH > ST) & (~is_land)
silver = silver.withColumn(
    "surface_habitable",
    when(too_big & (SH <= 1.2*ST), SH)  # keep small overshoot
    .when(too_big, None)
    .otherwise(SH)
)

# F) de-spike small integers
def null_small_nonland(c): 
    return when((~is_land) & (c < 10), None).otherwise(c)

silver = silver.withColumn("surface_totale",   null_small_nonland(col("surface_totale")))
silver = silver.withColumn("surface_habitable",null_small_nonland(col("surface_habitable")))

In [63]:
from pyspark.sql.functions import col, sum as Fsum, when

pairs = [
  ("property_type","type_de_bien"),
  ("etat","condition"),
  ("etage","etage_du_bien"),
  ("equipments","features_amenities_json"),
  ("annees","age_du_bien"),
  ("zoning","type_du_sol"),
]

for a,b in pairs:
    silver.groupBy("site").agg(
        Fsum(when(col(a).isNotNull(),1).otherwise(0)).alias(f"nn_{a}"),
        Fsum(when(col(b).isNotNull(),1).otherwise(0)).alias(f"nn_{b}"),
        Fsum(when(col(a).isNull() & col(b).isNotNull(),1).otherwise(0)).alias(f"fill_{a}_from_{b}"),
    ).show(truncate=False)

+-------+----------------+---------------+------------------------------------+
|site   |nn_property_type|nn_type_de_bien|fill_property_type_from_type_de_bien|
+-------+----------------+---------------+------------------------------------+
|mubawab|4853            |4853           |0                                   |
|avito  |5322            |0              |0                                   |
+-------+----------------+---------------+------------------------------------+

+-------+-------+------------+------------------------+
|site   |nn_etat|nn_condition|fill_etat_from_condition|
+-------+-------+------------+------------------------+
|mubawab|4508   |0           |0                       |
|avito  |0      |2001        |2001                    |
+-------+-------+------------+------------------------+

+-------+--------+----------------+-----------------------------+
|site   |nn_etage|nn_etage_du_bien|fill_etage_from_etage_du_bien|
+-------+--------+----------------+---------------

In [168]:
from pyspark.sql.functions import col, count

silver.groupBy("site", "property_type" , "type_de_bien") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").desc(), col("count").asc()) \
      .show(10, truncate=False)

+-------+------------------+----------------+-----+
|site   |property_type     |type_de_bien    |count|
+-------+------------------+----------------+-----+
|mubawab|Terrains et fermes|Ferme           |5    |
|mubawab|Villas et Riads   |Riad            |44   |
|mubawab|Maisons           |Maison          |75   |
|mubawab|Local commercial  |Local commercial|117  |
|mubawab|Bureaux           |Bureau          |125  |
|mubawab|Terrains et fermes|Terrain         |189  |
|mubawab|Villas et Riads   |Villa           |319  |
|mubawab|Appartements      |Appartement     |1467 |
|avito  |NULL              |NULL            |5    |
|avito  |Autre Immobilier  |NULL            |56   |
+-------+------------------+----------------+-----+
only showing top 10 rows



## condition

In [169]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "etat") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(10, truncate=False)


+-------+---------+-----+
|site   |etat     |count|
+-------+---------+-----+
|avito  |NULL     |3991 |
|mubawab|À rénover|60   |
|mubawab|NULL     |291  |
|mubawab|Nouveau  |806  |
|mubawab|Bon état |1184 |
+-------+---------+-----+



In [170]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "standing") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(10, truncate=False)


+-------+--------------+-----+
|site   |standing      |count|
+-------+--------------+-----+
|avito  |Economique    |30   |
|avito  |Moyen standing|70   |
|avito  |Haut standing |90   |
|avito  |NULL          |3801 |
|mubawab|NULL          |2341 |
+-------+--------------+-----+



In [171]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "condition") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(10, truncate=False)


+-------+---------+-----+
|site   |condition|count|
+-------+---------+-----+
|avito  |à rénover|32   |
|avito  |Bon état |344  |
|avito  |Neuf     |561  |
|avito  |NULL     |3054 |
|mubawab|NULL     |2341 |
+-------+---------+-----+



In [172]:
# --- Drop useless 'standing' column ---
if "standing" in silver.columns:
    silver = silver.drop("standing")

In [173]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, coalesce, trim
from pyspark.sql.types import StringType
import unicodedata, re

# ---------- helpers ----------

def _unaccent(s: str) -> str:
    if s is None:
        return None
    return "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )

def normalize_condition(v: str) -> str:
    if v is None:
        return None

    raw = v.strip()
    if raw == "":
        return None

    s = _unaccent(raw).strip().lower()
    s = re.sub(r"\s+", " ", s)

    # ---- mapping → ENGLISH ----
    # Bon état → Good condition
    if s in {"bon etat", "bon état"}:
        return "Good condition"

    # Neuf / Nouveau → New
    if s in {"neuf", "nouveau"}:
        return "New"

    # À rénover → To renovate
    if s in {"a renover", "à renover", "a rénover", "à rénover"}:
        return "To renovate"

    # empty / null-like
    if s in {"", "null", "none"}:
        return None

    # fallback: capitalized original (just in case new values appear)
    return raw.capitalize()

normalize_condition_udf = F.udf(normalize_condition, StringType())

# ---------- apply merge + normalization ----------

silver = silver.withColumn(
    "condition",
    normalize_condition_udf(
        trim(coalesce(col("condition"), col("etat")))
    )
)

# drop etat, we don't need it anymore
if "etat" in silver.columns:
    silver = silver.drop("etat")

# ---------- sanity check ----------

(
    silver.groupBy("site", "condition")
          .agg(F.count("*").alias("count"))
          .orderBy(col("site").asc(), col("count").asc())
          .show(50, truncate=False)
)


+-------+--------------+-----+
|site   |condition     |count|
+-------+--------------+-----+
|avito  |To renovate   |32   |
|avito  |Good condition|344  |
|avito  |New           |561  |
|avito  |NULL          |3054 |
|mubawab|To renovate   |60   |
|mubawab|NULL          |291  |
|mubawab|New           |806  |
|mubawab|Good condition|1184 |
+-------+--------------+-----+



## etage

In [174]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "etage") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)


+-------+-----+-----+
|site   |etage|count|
+-------+-----+-----+
|avito  |11   |1    |
|avito  |16   |2    |
|avito  |9    |2    |
|avito  |15   |3    |
|avito  |10   |7    |
|avito  |8    |8    |
|avito  |6    |27   |
|avito  |7    |28   |
|avito  |5    |73   |
|avito  |4    |128  |
|avito  |0    |154  |
|avito  |3    |175  |
|avito  |2    |257  |
|avito  |1    |339  |
|avito  |NULL |2787 |
|mubawab|NULL |2341 |
+-------+-----+-----+



In [175]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "etage_du_bien") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+-------------+-----+
|site   |etage_du_bien|count|
+-------+-------------+-----+
|avito  |NULL         |3991 |
|mubawab|13ème        |1    |
|mubawab|15ème        |1    |
|mubawab|18ème        |2    |
|mubawab|9ème         |6    |
|mubawab|10ème        |6    |
|mubawab|8ème         |8    |
|mubawab|7ème         |18   |
|mubawab|6ème         |22   |
|mubawab|5ème         |76   |
|mubawab|4ème         |145  |
|mubawab|1er          |217  |
|mubawab|3ème         |246  |
|mubawab|2ème         |250  |
|mubawab|NULL         |1343 |
+-------+-------------+-----+



In [176]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, coalesce
from pyspark.sql.types import StringType
import unicodedata, re

def _unaccent(s: str) -> str:
    if s is None:
        return None
    return "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )

def normalize_floor(etage_val: str, etage_bien_val: str) -> str:
    """
    Retourne un étage normalisé sous forme de string numérique ("0","1","2",...) ou None.
    Priorité : etage (Avito) puis etage_du_bien (Mubawab).
    """
    # 1) essayer d'utiliser etage (Avito)
    if etage_val is not None:
        raw = etage_val.strip()
        if raw != "":
            # si c'est déjà un nombre propre, on le garde
            if re.fullmatch(r"\d+", raw):
                return str(int(raw))  # normaliser genre "01" -> "1"
            # sinon on tente d'extraire des chiffres quand même
            m = re.search(r"\d+", raw)
            if m:
                return str(int(m.group(0)))

    # 2) fallback : etage_du_bien (Mubawab, ex: "3ème", "1er", "10ème")
    if etage_bien_val is not None:
        raw2 = etage_bien_val.strip()
        if raw2 != "":
            s = _unaccent(raw2).lower()
            s = re.sub(r"\s+", " ", s)

            # cas "RDC" possibles (au cas où ça arrive un jour)
            if s in {"rdc", "rez de chaussee", "rez-de-chaussee"}:
                return "0"

            # extraire le premier nombre qui passe (1er, 2eme, 3eme, 10eme, 18eme…)
            m2 = re.search(r"\d+", s)
            if m2:
                return str(int(m2.group(0)))

    # 3) sinon : rien
    return None

normalize_floor_udf = F.udf(normalize_floor, StringType())

# --------- appliquer fusion + normalisation ---------

silver = silver.withColumn(
    "etage",
    normalize_floor_udf(
        trim(col("etage")),
        trim(col("etage_du_bien"))
    )
)

# on n'a plus besoin d'etage_du_bien
if "etage_du_bien" in silver.columns:
    silver = silver.drop("etage_du_bien")

# --------- sanity check ---------
(
    silver.groupBy("site", "etage")
          .agg(F.count("*").alias("count"))
          .orderBy(col("site").asc(), col("count").asc())
          .show(100, truncate=False)
)

+-------+-----+-----+
|site   |etage|count|
+-------+-----+-----+
|avito  |11   |1    |
|avito  |9    |2    |
|avito  |16   |2    |
|avito  |15   |3    |
|avito  |10   |7    |
|avito  |8    |8    |
|avito  |6    |27   |
|avito  |7    |28   |
|avito  |5    |73   |
|avito  |4    |128  |
|avito  |0    |154  |
|avito  |3    |175  |
|avito  |2    |257  |
|avito  |1    |339  |
|avito  |NULL |2787 |
|mubawab|13   |1    |
|mubawab|15   |1    |
|mubawab|18   |2    |
|mubawab|10   |6    |
|mubawab|9    |6    |
|mubawab|8    |8    |
|mubawab|7    |18   |
|mubawab|6    |22   |
|mubawab|5    |76   |
|mubawab|4    |145  |
|mubawab|1    |217  |
|mubawab|3    |246  |
|mubawab|2    |250  |
|mubawab|NULL |1343 |
+-------+-----+-----+



## nombre_d_etage

In [177]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "nombre_d_etage") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+--------------+-----+
|site   |nombre_d_etage|count|
+-------+--------------+-----+
|avito  |4             |3    |
|avito  |3             |99   |
|avito  |2             |116  |
|avito  |1             |141  |
|avito  |NULL          |3632 |
|mubawab|NULL          |2341 |
+-------+--------------+-----+



In [178]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "nombre_d_etages") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+---------------+-----+
|site   |nombre_d_etages|count|
+-------+---------------+-----+
|avito  |NULL           |3991 |
|mubawab|4              |11   |
|mubawab|3              |71   |
|mubawab|2              |89   |
|mubawab|1              |95   |
|mubawab|NULL           |2075 |
+-------+---------------+-----+



In [179]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim
from pyspark.sql.types import StringType
import re

def normalize_nb_floors(v1: str, v2: str) -> str:
    """
    Normalise le nombre d'étages en chaîne numérique ("1","2",...)
    Priorité: v1 = nombre_d_etage (Avito), sinon v2 = nombre_d_etages (Mubawab).
    """
    # helper interne
    def _parse_num(x: str):
        if x is None:
            return None
        raw = x.strip()
        if raw == "":
            return None
        # si déjà un entier propre
        if re.fullmatch(r"\d+", raw):
            return str(int(raw))
        # sinon on tente au moins d'extraire des chiffres
        m = re.search(r"\d+", raw)
        if m:
            return str(int(m.group(0)))
        return None

    # 1) essayer Avito
    n1 = _parse_num(v1)
    if n1 is not None:
        return n1

    # 2) fallback Mubawab
    n2 = _parse_num(v2)
    if n2 is not None:
        return n2

    # 3) rien
    return None

normalize_nb_floors_udf = F.udf(normalize_nb_floors, StringType())

# --------- appliquer fusion + normalisation ---------

silver = silver.withColumn(
    "nombre_d_etages",
    normalize_nb_floors_udf(
        trim(col("nombre_d_etage")),
        trim(col("nombre_d_etages"))
    )
)

# on n'a plus besoin de la colonne Avito brute
if "nombre_d_etage" in silver.columns:
    silver = silver.drop("nombre_d_etage")

# --------- sanity check ---------

(
    silver.groupBy("site", "nombre_d_etages")
          .agg(F.count("*").alias("count"))
          .orderBy(col("site").asc(), col("count").asc())
          .show(100, truncate=False)
)

+-------+---------------+-----+
|site   |nombre_d_etages|count|
+-------+---------------+-----+
|avito  |4              |3    |
|avito  |3              |99   |
|avito  |2              |116  |
|avito  |1              |141  |
|avito  |NULL           |3632 |
|mubawab|4              |11   |
|mubawab|3              |71   |
|mubawab|2              |89   |
|mubawab|1              |95   |
|mubawab|NULL           |2075 |
+-------+---------------+-----+



## type 

In [182]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "type_d_appartement" , "type_de_bien") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+------------------+----------------+-----+
|site   |type_d_appartement|type_de_bien    |count|
+-------+------------------+----------------+-----+
|avito  |Duplex            |NULL            |41   |
|avito  |Studio            |NULL            |121  |
|avito  |NULL              |NULL            |3829 |
|mubawab|NULL              |Ferme           |5    |
|mubawab|NULL              |Riad            |44   |
|mubawab|NULL              |Maison          |75   |
|mubawab|NULL              |Local commercial|117  |
|mubawab|NULL              |Bureau          |125  |
|mubawab|NULL              |Terrain         |189  |
|mubawab|NULL              |Villa           |319  |
|mubawab|NULL              |Appartement     |1467 |
+-------+------------------+----------------+-----+



## terrain 

In [187]:

from pyspark.sql.functions import col, count

silver.filter(col("property_type") == "Terrains et fermes") \
      .groupBy("site", "type_de_terrain") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+--------------------------------------------------------------+-----+
|site   |type_de_terrain                                               |count|
+-------+--------------------------------------------------------------+-----+
|avito  |NULL                                                          |278  |
|mubawab|Groupement d'habitation, Lots de villa                        |1    |
|mubawab|Commercial, Groupement d'habitation, Industriel, Lots de villa|1    |
|mubawab|Agricole, Commercial, Lots de villa                           |1    |
|mubawab|Agricole, Commercial, Industriel                              |2    |
|mubawab|Agricole, Groupement d'habitation, Commercial, Lots de villa  |2    |
|mubawab|Groupement d'habitation, Lots de villa, Agricole              |2    |
|mubawab|Commercial, Industriel                                        |2    |
|mubawab|Agricole, Groupement d'habitation, Lots de villa              |2    |
|mubawab|Industriel                                 

In [196]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "zoning" , "property_type") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+--------------+------------------+-----+
|site   |zoning        |property_type     |count|
+-------+--------------+------------------+-----+
|avito  |Service public|Terrains et fermes|2    |
|avito  |NULL          |NULL              |5    |
|avito  |Industriel    |Terrains et fermes|10   |
|avito  |NULL          |Terrains et fermes|44   |
|avito  |Immeuble      |Terrains et fermes|50   |
|avito  |Villa         |Terrains et fermes|52   |
|avito  |NULL          |Autre Immobilier  |56   |
|avito  |Maison        |Terrains et fermes|57   |
|avito  |Agricole      |Terrains et fermes|63   |
|avito  |NULL          |Maisons           |118  |
|avito  |NULL          |Bureaux           |364  |
|avito  |NULL          |Local             |403  |
|avito  |NULL          |Villas et Riads   |431  |
|avito  |NULL          |Appartements      |2336 |
|mubawab|NULL          |Maisons           |75   |
|mubawab|NULL          |Local commercial  |117  |
|mubawab|NULL          |Bureaux           |125  |


In [197]:
from pyspark.sql import functions as F
import unicodedata, re

# --- utilitaires
def _unaccent(s):
    if not s:
        return None
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))

def normalize_zoning(value: str) -> str:
    if not value:
        return None
    s = _unaccent(value).lower().strip()
    s = re.sub(r'\s+', ' ', s)
    
    # découper si plusieurs zones séparées par des virgules
    parts = [p.strip() for p in s.split(",") if p.strip()]
    
    normalized = []
    for p in parts:
        if "agricol" in p:
            normalized.append("agricultural")
        elif "industriel" in p:
            normalized.append("industrial")
        elif "commercial" in p:
            normalized.append("commercial")
        elif "villa" in p:
            normalized.append("villa lots")
        elif "groupement" in p:
            normalized.append("residential cluster")
        elif "immeuble" in p:
            normalized.append("building")
        elif "maison" in p:
            normalized.append("house")
        elif "service public" in p:
            normalized.append("public service")
        else:
            normalized.append(p)
    
    # retirer doublons et re-joindre
    normalized = list(dict.fromkeys(normalized))
    return ", ".join(normalized) if normalized else None

normalize_zoning_udf = F.udf(normalize_zoning, "string")

# --- fusionner et normaliser
silver = (
    silver.withColumn("zoning", F.coalesce(F.col("zoning"), F.col("type_de_terrain")))
          .withColumn("zoning", normalize_zoning_udf(F.col("zoning")))
          .drop("type_de_terrain")
)

print("✅ zoning unified & normalized, type_de_terrain dropped.")
(
    silver.groupBy("zoning")
          .agg(F.count("*").alias("count"))
          .orderBy(F.col("count").desc())
          .show(50, truncate=False)
)


✅ zoning unified & normalized, type_de_terrain dropped.
+---------------------------------------------------------+-----+
|zoning                                                   |count|
+---------------------------------------------------------+-----+
|NULL                                                     |5920 |
|villa lots                                               |107  |
|agricultural                                             |85   |
|house                                                    |57   |
|building                                                 |50   |
|residential cluster                                      |39   |
|commercial                                               |26   |
|industrial                                               |13   |
|commercial, residential cluster                          |13   |
|residential cluster, commercial                          |4    |
|commercial, residential cluster, villa lots              |3    |
|agricultural, resid

## Age

In [ ]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "age_du_bien") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

In [ ]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "annees ") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

In [200]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, coalesce, trim
from pyspark.sql.types import StringType
import unicodedata, re

def _unaccent(s: str) -> str:
    if s is None:
        return None
    return "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )

def normalize_age(value: str) -> str:
    """
    Normalise l'âge du bien en anglais :
    - "Moins de 1 an", "Moins d'un an" -> "Less than 1 year"
    - "1-5 ans" -> "1-5 years"
    - "5-10 ans" -> "5-10 years"
    - "6-10 ans" -> "6-10 years"
    - "10-20 ans" -> "10-20 years"
    - "11-20 ans" -> "11-20 years"
    - "20-30 ans" -> "20-30 years"
    - "30-50 ans" -> "30-50 years"
    - "50-70 ans" -> "50-70 years"
    - "70-100 ans" -> "70-100 years"
    - "21+ ans" -> "More than 20 years"
    - "Plus de 100 ans" -> "More than 100 years"
    """
    if value is None:
        return None

    raw = value.strip()
    if raw == "":
        return None

    s = _unaccent(raw).lower()
    s = re.sub(r"\s+", " ", s)

    # < 1 an
    if s in {"moins de 1 an", "moins d'un an"}:
        return "Less than 1 year"

    # 1-5
    if s == "1-5 ans":
        return "1-5 years"

    # 5-10
    if s == "5-10 ans":
        return "5-10 years"

    # 6-10 (spécifique Avito)
    if s == "6-10 ans":
        return "6-10 years"

    # 10-20 (Mubawab)
    if s == "10-20 ans":
        return "10-20 years"

    # 11-20 (Avito)
    if s == "11-20 ans":
        return "11-20 years"

    # 20-30, 30-50, 50-70, 70-100
    if s == "20-30 ans":
        return "20-30 years"
    if s == "30-50 ans":
        return "30-50 years"
    if s == "50-70 ans":
        return "50-70 years"
    if s == "70-100 ans":
        return "70-100 years"

    # Avito: 21+ ans
    if s in {"21+ ans", "21 ans et plus"}:
        return "More than 20 years"

    # Mubawab: Plus de 100 ans
    if s == "plus de 100 ans":
        return "More than 100 years"

    # valeurs vides / null-like
    if s in {"", "null", "none"}:
        return None

    # fallback: retourner la version originale (trim)
    return raw

normalize_age_udf = F.udf(normalize_age, StringType())

# -------- 1) fusion age_du_bien + annees dans age_du_bien --------
silver = silver.withColumn(
    "age_du_bien",
    normalize_age_udf(
        trim(coalesce(col("age_du_bien"), col("annees")))
    )
)

# -------- 2) drop annees (on ne garde qu'une seule colonne) --------
if "annees" in silver.columns:
    silver = silver.drop("annees")

# -------- 3) sanity check --------
(
    silver.groupBy("site", "age_du_bien")
          .agg(F.count("*").alias("count"))
          .orderBy(col("site").asc(), col("count").asc())
          .show(100, truncate=False)
)


+-------+-------------------+-----+
|site   |age_du_bien        |count|
+-------+-------------------+-----+
|avito  |11-20 years        |15   |
|avito  |More than 20 years |20   |
|avito  |1-5 years          |28   |
|avito  |6-10 years         |28   |
|avito  |Less than 1 year   |68   |
|avito  |NULL               |3832 |
|mubawab|More than 100 years|2    |
|mubawab|70-100 years       |4    |
|mubawab|50-70 years        |4    |
|mubawab|30-50 years        |22   |
|mubawab|20-30 years        |48   |
|mubawab|10-20 years        |184  |
|mubawab|Less than 1 year   |246  |
|mubawab|5-10 years         |299  |
|mubawab|1-5 years          |466  |
|mubawab|NULL               |1066 |
+-------+-------------------+-----+



In [188]:
from pyspark.sql.functions import col, count

(
    silver
    .filter(col("property_type") == "Terrains et fermes") \
    .groupBy("site", "statut_du_terrain")
    .agg(count("*").alias("count"))
    .orderBy(col("site").asc(), col("count").asc())
    .show(100, truncate=False)
)


+-------+-----------------+-----+
|site   |statut_du_terrain|count|
+-------+-----------------+-----+
|avito  |NULL             |278  |
|mubawab|Non loti         |37   |
|mubawab|Loti             |72   |
|mubawab|NULL             |85   |
+-------+-----------------+-----+



In [189]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "constructibilite") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+----------------+-----+
|site   |constructibilite|count|
+-------+----------------+-----+
|avito  |NULL            |3991 |
|mubawab|R+9             |1    |
|mubawab|R+7             |2    |
|mubawab|Plain pied      |7    |
|mubawab|R+5             |8    |
|mubawab|R+4             |14   |
|mubawab|R+3             |18   |
|mubawab|R+1             |26   |
|mubawab|R+2             |32   |
|mubawab|NULL            |2233 |
+-------+----------------+-----+



In [201]:
silver.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- error: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- site: string (nullable = false)
 |-- offre: string (nullable = true)
 |-- price: decimal(38,0) (nullable = true)
 |-- title: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description_text: string (nullable = true)
 |-- surface_habitable: double (nullable = true)
 |-- caution: string (nullable = true)
 |-- zoning: string (nullable = true)
 |-- type_d_appartement: string (nullable = true)
 |-- surface_totale: double (nullable = true)
 |-- etage: string (nullable = true)


In [191]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "offre_match") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+-----------+-----+
|site   |offre_match|count|
+-------+-----------+-----+
|avito  |false      |5    |
|avito  |true       |3986 |
|mubawab|NULL       |2341 |
+-------+-----------+-----+



In [216]:
# Drop inutile column 'offre_match' from silver
silver = silver.drop("offre_match" , "caution" , "orientation" , "livraison" , "type_du_sol" , "type_de_bien")

# Vérification rapide
print("✅ Column 'offre_match' dropped successfully.")
silver.printSchema()

✅ Column 'offre_match' dropped successfully.
root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- error: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- site: string (nullable = false)
 |-- offre: string (nullable = true)
 |-- price: decimal(38,0) (nullable = true)
 |-- title: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description_text: string (nullable = true)
 |-- surface_habitable: double (nullable = true)
 |-- zoning: string (nullable = true)
 |-- type_d_appartement: string (nullable = true)
 |-- surface_totale: double (nullable = true)
 |-- etage: string (nullable = 

In [220]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "salons") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+-------------+-----+
|site   |disponibilite|count|
+-------+-------------+-----+
|avito  |3 mois       |24   |
|avito  |2 mois       |43   |
|avito  |+4 mois      |68   |
|avito  |Immédiate    |623  |
|avito  |NULL         |3233 |
|mubawab|NULL         |2341 |
+-------+-------------+-----+



In [222]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "salons") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+------+-----+
|site   |salons|count|
+-------+------+-----+
|avito  |8     |3    |
|avito  |638   |3    |
|avito  |6     |3    |
|avito  |7     |3    |
|avito  |5     |12   |
|avito  |4     |45   |
|avito  |0     |93   |
|avito  |3     |152  |
|avito  |2     |415  |
|avito  |NULL  |1505 |
|avito  |1     |1757 |
|mubawab|NULL  |2341 |
+-------+------+-----+



In [223]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "salle_de_bain") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+-------------+-----+
|site   |salle_de_bain|count|
+-------+-------------+-----+
|avito  |10           |2    |
|avito  |15           |2    |
|avito  |11           |3    |
|avito  |18           |3    |
|avito  |8            |5    |
|avito  |7            |20   |
|avito  |6            |27   |
|avito  |5            |56   |
|avito  |0            |114  |
|avito  |4            |135  |
|avito  |3            |267  |
|avito  |NULL         |593  |
|avito  |2            |1174 |
|avito  |1            |1590 |
|mubawab|NULL         |2341 |
+-------+-------------+-----+



In [224]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "chambres") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+--------+-----+
|site   |chambres|count|
+-------+--------+-----+
|avito  |20      |3    |
|avito  |9       |4    |
|avito  |10      |8    |
|avito  |11      |9    |
|avito  |7       |12   |
|avito  |8       |13   |
|avito  |6       |66   |
|avito  |5       |142  |
|avito  |4       |201  |
|avito  |1       |516  |
|avito  |3       |663  |
|avito  |NULL    |1167 |
|avito  |2       |1187 |
|mubawab|NULL    |2341 |
+-------+--------+-----+



In [225]:

from pyspark.sql.functions import col, count

silver.groupBy("site", "nombre_de_pieces") \
      .agg(count("*").alias("count")) \
      .orderBy(col("site").asc(), col("count").asc()) \
      .show(100, truncate=False)

+-------+----------------+-----+
|site   |nombre_de_pieces|count|
+-------+----------------+-----+
|avito  |9               |1    |
|avito  |16              |1    |
|avito  |50              |2    |
|avito  |10              |4    |
|avito  |8               |6    |
|avito  |6               |11   |
|avito  |4               |17   |
|avito  |5               |36   |
|avito  |2               |44   |
|avito  |3               |80   |
|avito  |1               |152  |
|avito  |NULL            |3637 |
|mubawab|NULL            |2341 |
+-------+----------------+-----+



In [231]:
from pyspark.sql.functions import col, count

(
    silver.filter(col("site") == "mubawab")
          .groupBy("site", "equipment" , "title")
          .agg(count("*").alias("count"))
          .orderBy(col("count").asc())
          .show(10, truncate=False)
)


+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+-----+
|site   |features_amenities_json                                                                                                                                                                                                                                                                                                                                            |title                                                                      |count|
+-------+-----------------------------------------------------------------------------------------------